# NLOptControl.jl

## Installation

There are several packages that need to be installed, to do this run:
```julia
Pkg.clone("https://github.com/JuliaMPC/PrettyPlots.jl")
Pkg.clone("https://github.com/JuliaMPC/VehicleModels.jl")
Pkg.clone("https://github.com/JuliaMPC/NLOptControl.jl")
```

Either [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) or [KNITRO.jl](https://github.com/JuliaOpt/KNITRO.jl) can be used for the nonlinear solver. Since **Ipopt** is free, all of the examples will use it and it can be added with:
```julia
Pkg.add("Ipopt");Pkg.build("Ipopt");
```

If you are using **Linux** make sure that you have **gfortran** to run **Ipopt**:
```julia
sudo apt-get install gfortran
```

Also, a plotting backend will be required and there are several options:
```julia
Pkg.add("PGFPlots");Pkg.build("PGFPlots")   # best looking
Pkg.add("GR");Pkg.build("GR");              # most reliable
Pkg.add("PyPlot");Pkg.build("PyPlot")       # also a good option  
```

## Packages that will be used

In [2]:
using NLOptControl, PrettyPlots
using Plots; pgfplots();           # this is optional, by default PrettyPlots uses gr() as a backend

## Quick Example: Brachistochrone
#### Solved by: John and Bernoulli, Newton and L'Hospital
### Given:
#### A particle sliding without friction along an unknown track in a gravitational feild
#### Dynamic Constraints
$$\dot{x}_1(t)=x_3(t)sin(u_1(t))$$ 
$$\dot{x}_2(t)=-x_3(t)cos(u_1(t))$$
$$\dot{x}_3(t)=gcos(u_1(t))$$

#### Boundary Conditions
$${x}_1(0)=0 \qquad {x}_1(t_f)=2$$
$${x}_2(0)=0\qquad {x}_2(t_f)=-2$$
$${x}_3(0)=0\qquad {x}_3(t_f)=free$$
### Find:
#### The track that minimizes time
$$J=t_f$$
### Solution: 

#### Differential Equations

Now we need to write all of the given information out. Let's start with the differential equation, that is written as an array of expressions:

In [3]:
de=[:(x3[j]*sin(u1[j])),:(-x3[j]*cos(u1[j])),:(9.81*cos(u1[j]))]

3-element Array{Expr,1}:
 :(x3[j] * sin(u1[j]))   
 :(-(x3[j]) * cos(u1[j]))
 :(9.81 * cos(u1[j]))    

Next let's write down the boundary conditions into an array:

In [4]:
X0=[0.0,0.0,0.0]
XF=[2.,-2.,NaN]

3-element Array{Float64,1}:
   2.0
  -2.0
 NaN  

#### Notice:
1. The numbers that where put into the expression are `Float64`; For now this is required!
2. The `NaN` is put into the boundary constraint for the third state; If any of the state bounds are free then pass a `NaN`

#### Define the Problem:
Now that we have the basic problem written down, we can call the `define()` function as:

In [5]:
n=define(de;numStates=3,numControls=1,X0=X0,XF=XF);

#### Basics
Variable | Description
:--- | :--- 
`n` | object that holds the entire optimal control problem 
`de` | array of differential equation expressions
`numStates` | the number of states
`numControls` | the number of controls
`X0` | intial state constraint
`XF` | final state constraint


Also, not in this problem, but

Variable | Description
:--- | :--- 
`XL` | lower state bound
`XU` | upper state bound
`CL` | lower state bound
`CU` | upper control bound

The above bounds can be set in the same fashion as the initial and final state constraints. i.e. in an Array.

#### Configure the Problem:
Now that the basic optimal control problem has been defined, the next step is to `configure!()` with additional options.

In [6]:
configure!(n;(:Nck=>[100]),(:finalTimeDV=>true));

#### Settings:
Key | Description
:--- | :--- 
`:Nck` | array of that holds the number of points within each interval
`:finalTimeDV` | bool to indicate if time is a design variable

#### Notice:
1. Final time is a design variable; we are trying to minimize it
2. We defined this as a single interval problem with ``100`` points

#### Objective Function
Finally, the objective function needs to be defined. For this, we use the ``JuMP`` macro ``@NLOptControl()`` directly as:

In [7]:
@NLobjective(n.mdl,Min,n.tf);

with,

Variable | Description
:--- | :--- 
`n.mdl` | object that holds them JuMP model 
`Min` | for a minimization problem
`n.tf` | a reference to the final time

#### Optimize
Now that the entire optimal control problem has been defined we can `optimize!()` it as:

In [8]:
optimize!(n);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



#### Post Process
**Make sure that you are not running the code in a folder where you have an important folder named `results`, because it will be deleted!**
Now that the problem has been optimized, we can quickly visualize the solution using ``allPlots()`` as:


In [9]:
allPlots(n)

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::IOStream) at ./deprecated.jl:30
 in _show(::IOStream, ::MIME{Symbol("image/png")}, ::Plots.Plot{Plots.PGFPlotsBackend}) at /home/febbo/.julia/v0.5/Plots/src/output.jl:223
 in show(::IOStream, ::MIME{Symbol("image/png")}, ::Plots.Plot{Plots.PGFPlotsBackend}) at /home/febbo/.julia/v0.5/Plots/src/output.jl:197
 in png(::Plots.Plot{Plots.PGFPlotsBackend}, ::String) at /home/febbo/.julia/v0.5/Plots/src/output.jl:8
 in savefig(::Plots.Plot{Plots.PGFPlotsBackend}, ::String) at /home/febbo/.julia/v0.5/Plots/src/output.jl:116
 in #statePlot#7(::Array{Any,1}, ::Function, ::NLOptControl.NLOpt, ::Int64, ::Int64) at /home/febbo/.julia/v0.5/PrettyPlots/src/NLOptControl_plots.jl:88
 in collect(::Base.Generator{UnitRange{Int64},PrettyPlots.##3#5{Int64,NLOptControl.NLOpt}}) at ./array.jl:307
 in #allPlots#2(::Int64, ::Function, ::NLOptControl.NLOpt) at /home/febbo/.julia/v0.5/PrettyPlots/src/NLOptControl_plots.jl:10
 in allPlots(::NLOp

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 Z M 2.484375 0 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.125 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 Z M 2.9375 -6.375 "/>
 
 
<path style="stroke:none;" d="M 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 L 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 Z M 1.265625 -0.765625 "/>
 
 
<path style="stroke:none;" d="M 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765625 C 1.5 -4.765625 1.703125 -4.984375 1.703125 -5.28125 C 1.703125 -5.765625 1.234375 -5.7

`allPlots()` automatically plots the solution to all of the state and control variables. In this problem, we may be interested in comparing two states against one another which can be done using the `statePlot()` function as:

In [10]:
st=statePlot(n,1,1,2)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 Z M 2.484375 0 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.125 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 Z M 2.9375 -6.375 "/>
 
 
<path style="stroke:none;" d="M 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 L 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 Z M 1.265625 -0.765625 "/>
 
 
<path style="stroke:none;" d="M 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765625 C 1.5 -4.765625 1.703125 -4.984375 1.703125 -5.28125 C 1.703125 -5.765625 1.234375 -5.7

For this case, there are four things that need to be passed to `statePlots()`:

Argument | Name | Description
:--- | :--- | :---
1|`n` |object that holds the entire optimal control problem
2|`idx` | reference to solution number used when we start solving `mpc` problems
3|`st1` | state number for `xaxis`
4|`st2` | state number for `yaxis`

#### Data Orginization
All of the states, control variables and time vectors are stored in an array of Dataframes called `n.r.dfs`

In [11]:
n.r.dfs

1-element Array{Any,1}:
 101×5 DataFrames.DataFrame
│ Row │ t           │ x1           │ x2           │ x3         │ u1          │
├─────┼─────────────┼──────────────┼──────────────┼────────────┼─────────────┤
│ 1   │ 0.0         │ -1.92482e-28 │ 0.0          │ 0.0        │ 3.7389e-19  │
│ 2   │ 0.000302536 │ 1.32471e-10  │ -4.48945e-7  │ 0.00296788 │ 0.000442609 │
│ 3   │ 0.0010139   │ 4.98629e-9   │ -5.04231e-6  │ 0.00994636 │ 0.00148333  │
│ 4   │ 0.00213113  │ 4.63039e-8   │ -2.2277e-5   │ 0.0209063  │ 0.00311783  │
│ 5   │ 0.00365302  │ 2.33209e-7   │ -6.54545e-5  │ 0.035836   │ 0.00534436  │
│ 6   │ 0.00557807  │ 8.30305e-7   │ -0.000152615 │ 0.0547203  │ 0.00816071  │
│ 7   │ 0.00790437  │ 2.36255e-6   │ -0.000306446 │ 0.0775401  │ 0.0115641   │
│ 8   │ 0.0106296   │ 5.74545e-6   │ -0.000554166 │ 0.104272   │ 0.0155511   │
│ 9   │ 0.0137511   │ 1.24386e-5   │ -0.00092738  │ 0.13489    │ 0.0201179   │
│ 10  │ 0.0172658   │ 2.46206e-5   │ -0.00146191  │ 0.16936    │ 0.0252599   │


It is an array because the problem is designed to be solved multiple times in a receding time horizon. The variables can be accesed like this:

In [12]:
n.r.dfs[1][:x1][1:4]

4-element DataArrays.DataArray{Float64,1}:
 -1.92482e-28
  1.32471e-10
  4.98629e-9 
  4.63039e-8 

#### Optimization Data

In [13]:
n.r.dfs_opt

1-element Array{Any,1}:
 1×4 DataFrames.DataFrame
│ Row │ t_solve │ obj_val  │ status  │ iter_num │
├─────┼─────────┼──────────┼─────────┼──────────┤
│ 1   │ 5.97999 │ 0.824339 │ Optimal │ 0        │

The sailent optimization data is stored in the table above

Variable | Description
:--- | :--- 
`t_solve` | cpu time for optimization problem
`obj_val` | objective function value
`iter_num` | a variable for a higher-level algorithm, often these problems are nested

One thing that may be noticed is the long time that it takes to solve the problem. This is typical for the first optimization, but after that even if the problem is modified the optimization time is greatly reduced. 

#### For instance, let's re-run the optimization:

In [14]:
optimize!(n);

In [15]:
n.r.dfs_opt[2][:t_solve]

1-element DataArrays.DataArray{Float64,1}:
 0.167305

#### Since the problem is hot-started it can solve much faster!

## Quick Example: Moon Lander

### Given:
#### A space-ship landing on the moon 
#### Dynamic Constraints
$$\dot{x}_1(t)=x_2(t)$$ 
$$\dot{x}_2(t)=u(t)-g$$

#### Boundary Conditions
$${x}_1(0)=10 \qquad {x}_1(t_f)=0$$
$${x}_2(0)=-2\qquad {x}_2(t_f)=0$$

#### Control Limits
$${u}_{1_{min}}=0 $$
$${u}_{1_{max}}=3 $$

### Find:
#### The track that minimizes time
$$J=\int_{0}^{tf} u(t) dt$$
### Solution:
In this problem, we put the bounds directly into `define()`. Also, now we have constant limits on the control variables and those can be added as shown below

In [16]:
de=[:(x2[j]),:(u1[j]-1.625)]
n=define(de;numStates=2,numControls=1,X0=[10.,-2],XF=[0.,0.],CL=[0.],CU=[3.]);
configure!(n,Nck=[10,10,10,10];(:finalTimeDV=>true));

#### Optional state and control variable descriptions
The state and control variables are by default, $:x1,:x2,..$ and $:u1,:u2,..$, but they can be changed for the plots, **i.e. n.dfs** with the following commands:

In [17]:
names=[:H,:V]; descriptions = ["Height(t)","Velocity(t)"]; stateNames!(n,names,descriptions);
names=[:T]; descriptions = ["Thrust(t)"]; controlNames!(n,names,descriptions);

Next, now that the problem is configured, all of the state and control variables are stored in JuMP Arrays, `n.r.x[:,:]` and `n.r.u[:,:]`, respectively. For instance;

In [18]:
typeof(n.r.x)

Array{JuMP.Variable,2}

#### Integral Terms in the Cost Function
`integrate!()` is used to make terms that can be added to the cost function that need to be integrated. When calling this function the key variable must be set to either `:state` or `:control`

`:variable` | Descriptions
:--- | :--- 
`:state` | indicates a state variable will be integrated
`:control` | indicates a control variable will be integrated

Currently there are several forms that these integrals can take

`:integrate` | Description
:--- | :---
`:default`| basic integration of variable
`:squared` | integrates the square of the variable
`:cos` | integrates the cos of the variable
`:sin` | integrates the sin of the variable


In this example the first control variable `n.r.u[:,1]`needs to be integrated. To do this, `integrate!()` can be used as: 

In [19]:
obj=integrate!(n,n.r.u[:,1];(:variable=>:control));

#### Now this term can be added as the objective function and the problem can be solved

In [20]:
@NLobjective(n.mdl, Min, obj);
optimize!(n);

#### Optional plot settings
Many of the plot settings can be modified using the `plotSettings()` function. For instance;

In [21]:
plotSettings(;(:mpc_lines =>[(4.0,:red,:solid)]),(:size=>(700,700)));
allPlots(n)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 Z M 2.484375 0 "/>
 
 
<path style="stroke:none;" d="M 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 Z M 2.9375 -6.375 "/>
 
 
<path style="stroke:none;" d="M 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 L 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 Z M 1.265625 -0.765625 "/>
 
 
<path style="stroke:none;" d="M 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765625 C 1.5 -4.765625 1.703125 -4.984375 1.703125 -5.28125 C 1.703125 -5.765625 1.234375 -5.765625 1.09375 -5.765625 C 1.390625 -6.265625 2.046875 -6.390625 2.40625 -6.390625 C 2.828125 -6.390625 3.375 -6.171875 3.375 -5.28125 C 3.375 -5.15625 3.34375 -4.578125 3.09375 -4.140625 C 2.796875 -3.65625 2.453125 -3.625 2.203125 -3.625 C 2.125 -3.609375 1.890625 -3.59375 1.8125 -3.59375 C 1.734375 -3.578125 1.671875 -3.5625 1.671875 -3.46875 C 1.671875 -3.359375 1.734375 -3.359375 1.90625 -3.359375 L 2.34375 -3.359375 C 3.15625 -3.359375 3.53125 -2.6875 3.53125 -1.703125 C 3.53125 -0.34375 2.84375 -0.0625 2.40625 -0.0625 C 1.96875 -0.0625 1.21875 -0.234375 0.875 -0.8125 C 1.21875 -0.765625 1.53125 -0.984375 1.53125 -1.359375 C 1.53125 -1.71875 1.265625 -1.921875 0.984375 -1.921875 C 0.734375 -1.921875 0.421875 -1.78125 0.421875 -1.34375 C 0.421875 -0.4375 1.34375 0.21875 2.4375 0.21875 C 3.65625 0.21875 4.5625 -0.6875 4.5625 -1.703125 C 4.5625 -2.515625 3.921875 -3.296875 2.890625 -3.515625 Z M 2.890625 -3.515625 "/>
 
 
<path style="stroke:none;" d="M 2.9375 -1.640625 L 2.9375 -0.78125 C 2.9375 -0.421875 2.90625 -0.3125 2.171875 -0.3125 L 1.96875 -0.3125 L 1.96875 0 C 2.375 -0.03125 2.890625 -0.03125 3.3125 -0.03125 C 3.734375 -0.03125 4.25 -0.03125 4.671875 0 L 4.671875 -0.3125 L 4.453125 -0.3125 C 3.71875 -0.3125 3.703125 -0.421875 3.703125 -0.78125 L 3.703125 -1.640625 L 4.6875 -1.6406

## Other Dynamic Constraint Methods
Currently there are three different methods to ensure that the dyanamic constraints are satisfied and they are set when `configure!()` is called using the `:integrationScheme` key. They are listed below:

`:integrateScheme` | Description
:--- | :---
`:lgrExplicit`| default scheme; implementation of hp-pseudospecral method
`:bkwEuler` | approximate using backward euler method
`:trapezoidal` | approximate using trapezoidal method

The later two are time-marching methods and default number of points is `100`, but that can be changed by setting `N`. So, the above problem can be solved using one of the time-marching schemes as: 

In [ ]:
n=define(de;numStates=2,numControls=1,X0=[10.,-2],XF=[0.,0.],CL=[0.],CU=[3.]);
configure!(n,N=200;(:integrationScheme=>:trapezoidal),(:finalTimeDV=>true));
obj=integrate!(n,n.r.u[:,1];(:variable=>:control));
@NLobjective(n.mdl, Min, obj);
optimize!(n);
allPlots(n)

## Constraints
Often when building a model and using it to solve an optimal control problem, their are issues associated with infeasibility. `NLOptControl` has functionality to help deal with these issues. For instance, the dual infeasibility values can stored and quickly viewed. They are stored in an array of DataFrames which can be referenced with `n.r.dfs_con` as:  

In [ ]:
n.r.dfs_con

It is empty, because by default this data is not calculated and stored. This option can be turned on by modifying the settings for the problem:

In [ ]:
n.s.evalConstraints

In [ ]:
n.s.evalConstraints=true;
optimize!(n);
n.r.dfs_con

In [ ]:
evalMaxDualInf(n)

The last function called, searches through all of the `dual infeasibilities` to find the largest value. 
As, this problem is, it is feasible and optimal. But if there was an issue, often looking for high values in these DataFrame structures is the quickest way to figure out the constraints that are giving the solver trouble. 
## Tolerances
If there was an example where the `dual infeasibility` value for one or more of the variables was very high, but the actual constraint is only being violated slightly (by some reasonable amount) then the tolerances on the initial and terminal states can be adjusted. This will also improve the solve time, so it is good practice to set these to reasonable values. For instance, in the `Moon Lander` example, we can set them as:

In [ ]:
n=define(de;numStates=2,numControls=1,X0=[10.,-2],XF=[0.,0.],CL=[0.],CU=[3.]);

XF_tol=[2.0,0.5];
X0_tol=[0.05,0.05];
defineTolerances!(n;X0_tol=X0_tol,XF_tol=XF_tol);

configure!(n,N=50;(:integrationScheme=>:bkwEuler),(:finalTimeDV=>true));
obj=integrate!(n,n.r.u[:,1];(:variable=>:control));
@NLobjective(n.mdl, Min, obj);
optimize!(n);
allPlots(n) 

## Hands-On Example: Bryson Denham
Double integrator problem

### Given:

#### Dynamic Constraints
$$\dot{x}_1(t)=x_2(t)$$ 
$$\dot{x}_2(t)=u_1(t)$$

#### Boundary Conditions
$${x}_1(0)=0 \qquad {x}_1(t_f)=0$$
$${x}_2(0)=1\qquad {x}_2(t_f)=-1$$

#### State Limit
$${x}_{1_{max}}=1/6$$

### Find:
#### The track that minimizes time
$$J=\int_{0}^{1} u(t)^2 dt$$

### Solution: 

In [ ]:
de=[:(x2[j]),:(u1[j])]
n=define(de;numStates=2,numControls=1,X0=[0.,1],XF=[0.,-1.],XL=[0.,NaN],XU=[1/9,NaN],CL=[NaN],CU=[NaN]);
configure!(n;(:finalTimeDV=>false),(:tf=>1.0));
obj=integrate!(n,n.r.u[:,1];C=0.5,(:variable=>:control),(:integrand=>:squared));
@NLobjective(n.mdl,Min,obj);
optimize!(n);
allPlots(n)

## Hands-On Example: Rocket Problem
#### ref: https://github.com/JuliaOpt/juliaopt-notebooks/blob/master/notebooks/JuMP-Rocket.ipynb

### Given:
#### A rocket with drag force losing mass from burning fuel excaping a gravitational feild
#### Dynamic Constraints

$$\dot{x}_1(t)=x_2(t)$$ 
$$\dot{x}_2(t)=(u_1(t)-Drag(t))/x_3(t)-Grav(t)$$
$$\dot{x}_3(t)=-u_1(t)/c$$

with,
$$Drag(t)=D_cx_2(t)^2\exp(-h_c(x_1(t)-h_0)/h_0)$$
$$Grav(t)=g_0(h_0/x_1(t))^2$$

where,
1. $x_1$ is the height
2. $x_2$ is the velocity
3. $x_3$ is the mass
4. $u_1$ is the thrust

#### Boundary Conditions
$${x}_1(0)=h_0\qquad {x}_1(t_f)=free$$
$${x}_2(0)=v_0\qquad {x}_2(t_f)=free$$
$${x}_3(0)=m_0\qquad {x}_3(t_f)=m_f$$

#### State Limits
$${x}_{1_{min}}=h_0\qquad {x}_{1_{max}}=free$$
$${x}_{2_{min}}=v_0\qquad {x}_{1_{max}}=free$$
$${x}_{3_{min}}=m_f\qquad {x}_{1_{max}}=m_0$$

#### Control Limits
$${u}_{1_{min}}=0.0\qquad {u}_{1_{max}}=T_{max}$$

### Find:
#### The track that minimizes rocket height
$$J=x_3(t_f)$$
### Solution: 


In [ ]:
# Constants
h_0 = 1.    # Initial height
v_0 = 0.    # Initial velocity
m_0 = 1.    # Initial mass
g_0 = 1.    # Gravity at the surface

# Parameters
T_c = 3.5  # Used for thrust
h_c = 500  # Used for drag
v_c = 620  # Used for drag
m_c = 0.6  # Fraction of initial mass left at end

# Derived parameters
c     = 0.5*sqrt(g_0*h_0)  # Thrust-to-fuel mass
m_f   = m_c*m_0            # Final mass
D_c   = 0.5*v_c*m_0/g_0    # Drag scaling
T_max = T_c*g_0*m_0        # Maximum thrust


Grav=:($g_0*($h_0/x1[j])^2);
Drag=:()  #TODO

de=Array{Expr}(3,);
de[1]=:(x2[j]);
#TODO finish!